In [3]:
import copy
import glob
import random
import regex as re
import numpy as np
import pandas as pd
import datetime as dt
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline
%load_ext rpy2.ipython
%run ../airquality/data/gen_daily_targets.py
%run ../airquality/data/prepare_data.py

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
data = pd.read_csv('../data/data_roll_day_dist.csv', index_col=0)

In [28]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import log_loss, make_scorer

lags = 25
X = data.loc[pd.to_datetime(data['day']) < '2015-01-01'].drop(['station', 'day', 'date'], axis=1)
y = data.loc[pd.to_datetime(data['day']) < '2015-01-01', 'conc_obs'] > 100

params = pd.DataFrame({
    'eta': random.sample(list(np.linspace(0.01, 2, 201)), 200),
    'n_estimators': random.sample(list(range(1, 250)), 200),
    'reg_alpha': random.sample(list(np.linspace(0.01, 1, 201)), 200),
    'max_depth': random.sample(list(range(1, 35)), 25) * 8,
    'score': [np.nan] * 200
})
for i, r in params[['eta', 'n_estimators', 'reg_alpha', 'max_depth']].head(10).iterrows():
    kf = KFold(n_splits=3)
    metric = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        p = dict(r)
        p['n_estimators'] = int(p['n_estimators'])
        p['max_depth'] = int(p['max_depth'])
        p['n_jobs'] = 4
        model = xgb.XGBClassifier(**p)
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        metric.append(
            log_loss(y_test, y_pred, labels=(0, 1))
        ) 

    params.loc[i, 'score'] = np.mean(metric)
    print(params.iloc[i])

params.to_csv('../reports/cv_xgb_Jonas_6_{}_{}.csv'
              .format(s, len(glob.glob('../reports/cv_xgb_Jonas_6_{}_*'.format(s)))))
params.sort_values('score').head()

eta             1.32
max_depth      25.00
n_estimators   78.00
reg_alpha       0.23
score           0.00
Name: 0, dtype: float64
eta              0.61
max_depth       26.00
n_estimators   249.00
reg_alpha        0.83
score            0.00
Name: 1, dtype: float64
eta             1.03
max_depth      12.00
n_estimators   54.00
reg_alpha       0.32
score           0.00
Name: 2, dtype: float64


KeyboardInterrupt: 

In [29]:
X = data.loc[pd.to_datetime(data['day']) < '2015-01-01'].drop(['station', 'day', 'date'], axis=1)
y = data.loc[pd.to_datetime(data['day']) < '2015-01-01', 'conc_obs'] > 100
X = data.loc[pd.to_datetime(data['day']) < '2015-01-01'].drop(['station', 'day', 'date'], axis=1)
y = data.loc[pd.to_datetime(data['day']) < '2015-01-01', 'conc_obs'] > 100
X_train = X.loc[X['day'] < '2015-01-01'].drop(['station', 'day', 'date'], axis=1)
y_train = y.loc[X['day'] < '2015-01-01']
X_test = X.loc[X['day'] >= '2015-01-01'].drop(['station', 'day', 'date'], axis=1)
y_test = y.loc[X['day'] >= '2015-01-01']

model = xgb.XGBClassifier(e)
model.fit(X_train, y_train)
model.predict(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'loc'

In [ ]:
y_pred = model.predict_proba(X_test)
y_test['pred'] = y_pred[:, 1]
submission = pd.merge(test, y_test[['pred']].reset_index().rename(columns={'day': 'date'}), on=['date', 'station'])\
    .drop('target', axis=1)
                      
submission[['pred']].to_csv('../reports/submission3.csv', header=True, index=False)
submission.head()

In [254]:
%cat ../reports/submission1.csv

pred
0.9784955382347107
0.07289516180753708
0.768291175365448
0.2723965048789978
0.03478242829442024
0.21929146349430084
0.6432662010192871
0.007857130840420723
0.05543694645166397
0.4935362637042999
0.8628844618797302
0.5083826184272766
0.4356672763824463
0.677058756351471
0.0076334127224981785
0.35420575737953186
0.39222782850265503
0.8895682096481323
0.5083826184272766
0.29070043563842773
0.17258630692958832
0.007857130840420723
0.07289516180753708
0.39222782850265503
0.8040553331375122
0.0785490944981575
0.31986910104751587
0.6087238788604736
0.007857130840420723
0.05543694645166397
0.32117849588394165
0.8895682096481323
0.0785490944981575
0.29070043563842773
0.6432662010192871
0.007104177493602037
0.05543694645166397
0.32117849588394165
0.2723965048789978
0.07414732873439789
0.013499737717211246
0.0786505863070488
0.003322285832837224
0.012142645195126534
0.32117849588394165
0.20981541275978088
0.03478242829442024
0.02264479175209999
0.1199246421456337
0.003322285832837224
0.01214

In [301]:
wide_series_imputed_xgb = pd.read_csv('../data/wide_series_imputed_xgb.csv', index_col=0).fillna(method='ffill')
wide_series_imputed_xgb.head()

,ES0691A_mean,ES1396A_mean,ES1438A_mean,ES1480A_mean,ES1679A_mean,ES1856A_mean,ES1992A_mean,ES0691A_max,ES1396A_max,ES1438A_max,...,ES1679A_max,ES1856A_max,ES1992A_max,ES0691A_std,ES1396A_std,ES1438A_std,ES1480A_std,ES1679A_std,ES1856A_std,ES1992A_std
day,,,,,,,,,,,,,,,,,,,,,
2013-01-01,42.39,38.61,51.39,43.87,37.13,14.78,23.35,72.00,98.00,112.00,...,69.00,39.00,68.00,18.13,26.67,25.73,24.63,20.29,10.99,21.15
2013-01-02,52.50,47.58,63.71,54.50,48.75,32.08,43.29,87.00,86.00,119.00,...,76.00,77.00,85.00,16.61,24.72,26.48,19.18,16.91,25.43,26.53
2013-01-03,64.62,54.62,74.04,64.17,49.25,38.88,49.04,119.00,85.00,117.00,...,78.00,94.00,110.00,22.69,19.05,22.87,21.70,14.87,22.17,31.43
2013-01-04,54.12,44.30,44.50,89.13,61.92,22.18,39.04,80.00,113.00,123.00,...,108.00,60.00,99.00,16.58,28.35,32.46,45.41,25.83,16.98,28.20
2013-01-05,36.88,39.17,44.50,89.00,64.00,12.25,33.54,60.00,114.00,123.00,...,114.00,16.00,85.00,11.00,22.21,32.46,21.63,17.04,2.56,17.75


In [337]:
tall_series = wide_series_imputed_xgb[[s + '_max' for s in test.station.unique()]].reset_index().melt(id_vars='day')
tall_series['station'] = tall_series.variable.apply(lambda v: v[:7])
tall_series = tall_series.drop('variable', axis=1)
tall_series = tall_series.join(pd.get_dummies(tall_series['station']))

X = tall_series
y = tall_series['value'] > 100

X_train = X.loc[X['day'] < '2015-01-01'].drop(['station', 'day'], axis=1)
y_train = y.loc[X['day'] < '2015-01-01']
X_test = X.loc[X['day'] >= '2015-01-01'].drop(['station', 'day'], axis=1)
y_test = y.loc[X['day'] >= '2015-01-01']

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
model.predict(X_test)

array([False,  True, False, ..., False,  True, False], dtype=bool)